In [46]:
import pandas as pd

from unidecode import unidecode

In [47]:
mdf = pd.read_csv('musicas.csv')
mdf.sample(5)

,indice_musica,titulo_musica,nome_artista,artistas_participacao,letra,album,data_lancamento,tags,identificador
913,913,Molhado De Suor,Alceu Valença,[],Eu gosto\nÉ de ter ver bonita\nCom aquele vest...,molhado de suor (1974),1974,"Country,Rock,MPB (Música Popular Brasileira),L...",Molhado De Suor Alceu Valença
492,492,Eu Te Avisei,Tuyo,['Terno\xa0Rei'],"[Letra de ""Eu Te Avisei"" com Tuyo & Terno Rei]...",NaN,"Mar. 27, 2020","Pop,MPB (Música Popular Brasileira),Brasil,Em ...",Eu Te Avisei Tuyo
17,17,Pra Não Dizer Que Não Falei das Flores,Geraldo Vandré,[],"[Letra de ""Pra Não Dizer Que Não Falei das Flo...",Pra Não Dizer Que Não Falei das Flores (1979),1968,"Pop,Live,Singer-Songwriter,Folk,Contemporary F...",Pra Não Dizer Que Não Falei das Flores Geraldo...
547,547,Jurema,Tim Maia,[],"[Letra de ""Jurema""]\n\n[Verso]\nSome don't thi...",Tim Maia (1970) (1970),"Jan. 5, 1970","Pop,MPB (Música Popular Brasileira),Brasil,Em ...",Jurema Tim Maia
318,318,Hater,Carne Doce,[],"[Letra de ""Hater"" com Carne Doce]\n\n[Verso 1]...",Interior (2020),"Sep. 15, 2020","Rock,Experimental,Alternative,Brasil,Em Portug...",Hater Carne Doce


In [3]:
# Quantidade de artistas distintos no CSV

artistas_distintos
mdf['nome_artista'].nunique()

245

In [45]:
mdf['artistas_participacao'].unique()

array(["['Tiago\\xa0Iorc']", '[]', "['Milton\\xa0Nascimento']",
       "['Carlinhos\\xa0Brown']", "['MPB4']",
       "['Charli\\xa0XCX', 'Diplo']", "['Jorge\\xa0Ben Jor']",
       "['Caetano\\xa0Veloso']", "['Bruno\\xa0Martini']",
       "['Letrux', 'Liniker', 'Luedji\\xa0Luna', 'Xênia\\xa0França']",
       "['Rafael\\xa0Fantini']",
       "['Devendra\\xa0Banhart', 'Rodrigo\\xa0Amarante']",
       "['Vinícius\\xa0de Moraes']", "['Seu\\xa0Jorge']", "['LUDMILLA']",
       "['Beto\\xa0Guedes']", "['Nando\\xa0Reis']",
       "['Mariana\\xa0Volker']",
       "['Gui\\xa0Amabis', 'Juçara\\xa0Marçal', 'Rodrigo\\xa0Campos']",
       "['Alaíde\\xa0Costa']", "['Arnaldo\\xa0Antunes']",
       "['ANAVITÓRIA']", "['Ivete\\xa0Sangalo']", "['Roupa\\xa0Nova']",
       "['LAZÚLI\\xa0(BRA)']", "['Lucas\\xa0Silveira']",
       "['Bixiga\\xa070']", "['Ashira']", "['Abacaxepa']",
       "['Ed\\xa0Motta']", "['Tropkillaz']", "['Gilberto\\xa0Gil']",
       "['BaianaSystem', 'Virgínia\\xa0Rodrigues']", "['DUDA

In [14]:
# Função que recebe um nome e retorna o link do perfil do artista correspondente no Genius 

def get_link_artista(nome_artista):
    nome_normalizado = unidecode(nome_artista).lower().capitalize().replace(' ', '-')
    link_perfil = f'https://genius.com/artists/{nome_normalizado}'
    
    return link_perfil

In [42]:
# Criando um DF apenas com o nome dos artistas distintos existentes

df_artistas = pd.DataFrame(mdf['nome_artista'].unique(), columns = ['nome_artista'])
df_artistas

,nome_artista
0,ANAVITÓRIA
1,Milton Nascimento
2,Chico Buarque
3,Silva & Anitta
4,Zé Ramalho
...,...
240,Criolo & Ney Matogrosso
241,¡IT’S ALESSANDRO!
242,Gonzaguinha & Maria Rita
243,Marcelo Jeneci


In [43]:
# Adicionando o link do perfil de cada artista no Genius 

df_artistas['link_perfil'] = df_artistas['nome_artista'].apply(get_link_artista)
df_artistas

,nome_artista,link_perfil
0,ANAVITÓRIA,https://genius.com/artists/Anavitoria
1,Milton Nascimento,https://genius.com/artists/Milton-nascimento
2,Chico Buarque,https://genius.com/artists/Chico-buarque
3,Silva & Anitta,https://genius.com/artists/Silva-&-anitta
4,Zé Ramalho,https://genius.com/artists/Ze-ramalho
...,...,...
240,Criolo & Ney Matogrosso,https://genius.com/artists/Criolo-&-ney-matogr...
241,¡IT’S ALESSANDRO!,https://genius.com/artists/!it's-alessandro!
242,Gonzaguinha & Maria Rita,https://genius.com/artists/Gonzaguinha-&-maria...
243,Marcelo Jeneci,https://genius.com/artists/Marcelo-jeneci
